In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from PIL import Image, ImageTk
import os
import sys
import glob
from google.colab.patches import cv2_imshow

Merging requires a path to folder, where predicted images are stored.

In [36]:
def merge(pth):
  if(not os.path.exists(pth)):
    print("This path does not exist.")
  if(os.path.exists(pth)):
    #creates an empty list, where all file names of .png images from the input folder will be stored
    filenames = []
    #creates an empty list of column and row values
    cols = []
    rows = []
    #counts, how many files are in the input folder
    path, dirs, files = next(os.walk(pth))
    png_extension = ".png"
    #saves file names of .png images in the list
    for i in range (0,len(files)):
      if png_extension in files[i]:
        filenames.append(files[i])
    #print(filenames)
    for j in range (0,len(filenames)):
      cols.append(int(files[j][3:6]))
      rows.append(int(files[j][9:12]))
    #gets the number of columns and rows
    colmax = max(cols)
    rowmax = max(rows)
    dims = colmax * rowmax
    print(colmax, rowmax, dims)
    #creates an empty image, where the predictions will be put into
    width = 208 * (colmax+1)
    height = 208 * (rowmax+1) 
    img = np.zeros((width,height,3), np.uint8)
    #cv2_imshow(img)
    #adds image to region of interest
    for k in range(0,colmax):
      for l in range(0,rowmax):
        ppath = "content/grdive/MyDrive/Projekty/UGS_Klinove_pismo/Data/predictions/tablet_figure_cuts_results_yolo/exp4/vector_preds/"
        fname = f'Col{k:03d}Row{l:03d}_output.png'
        path_to_file = os.path.join(path,fname)
        if(os.path.exists(path_to_file)):
          print(path_to_file)
          small_img = cv2.imread(path_to_file)
          small_img = cv2.resize(small_img,(416, 416))
          #cv2_imshow(small_img)    
          y_min = (k-1) * 208
          y_max = (k-1) * 208 + 416
          x_min = (l-1) * 208
          x_max = (l-1) * 208 + 416
          if y_min < 0:
            y_min = 0
          if x_min < 0:
            x_min = 0
          if y_max > height:
            y_min = height - 416
            y_max = height
          if x_max > width:
            x_min = width - 416
            x_max = width
          x_length = x_max - x_min
          y_width = y_max - y_min
          roi = img[y_min:y_max, x_min:x_max]
          small_img_gray = cv2.cvtColor(small_img, cv2.COLOR_RGB2GRAY)
          ret, mask = cv2.threshold(small_img_gray, x_length, y_width, cv2.THRESH_BINARY)
          bg = cv2.bitwise_or(roi,roi,mask = mask)
          mask_inv = cv2.bitwise_not(small_img_gray)
          try:
            fg = cv2.bitwise_and(small_img,small_img, mask=mask_inv)
            
            print("roi")
            cv2_imshow(roi)
            print("fg")
            cv2_imshow(fg)
            print("mask_inv")
            cv2_imshow(mask_inv)

            #final_roi = cv2.add(bg, fg)
            final_roi = np.zeros((416,416,3), np.uint8)
            final_roi = cv2.add(roi, fg)
            
            print("final_roi")
            cv2_imshow(final_roi)
            
            small_img = final_roi
            img[y_min : y_min + small_img.shape[0], x_min : x_min + small_img.shape[1]]= small_img
          except:
            e = sys.exc_info()[0]
            print(e)
            pass
    cv2_imshow(img)
    

    

In [37]:
merge('/content/gdrive/MyDrive/Projekty/UGS_Klinove_pismo/Data/predictions/tablet_figure_cuts_results_yolo/exp4/vector_preds/')

Output hidden; open in https://colab.research.google.com to view.